In [ ]:
!pip install -U transformers datasets accelerate bitsandbytes peft trl 

In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["WANDB_DISABLED"] = "true"


def set_deterministic(seed=42):
    """Установка всех случайных seed для детерминированности"""
    # Python
    random.seed(seed)
    
    # Numpy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # для multi-GPU
    
    # Установка детерминированных алгоритмов
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Дополнительные настройки для CuDNN
    torch.backends.cudnn.enabled = False
    
    # Установка переменной окружения
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.use_deterministic_algorithms(True)

set_deterministic()

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_name = "/kaggle/input/qwen-3/transformers/4b-base/1"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

r = 8
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
SYSTEM_PROMPT = (
    "You are the actual assistant. Rules:\n"
    "1) Answer format: answer\n"
    "2) Don't make things up, don't fantasize, don't fill in missing information.\n"
    "3) Write in Russian, without quotation marks or trailing periods.\n"
    "4) One answer, one line, maximum 128 characters per answer.\n"
    '5) If the question doesn not have an answer for some reason, answer: Я не знаю.'
) 

In [5]:
print(SYSTEM_PROMPT)

You are the actual assistant. Rules:
1) Answer format: answer
2) Don't make things up, don't fantasize, don't fill in missing information.
3) Write in Russian, without quotation marks or trailing periods.
4) One answer, one line, maximum 128 characters per answer.
5) If the question doesn not have an answer for some reason, answer: Я не знаю.


In [27]:
import json
import pandas as pd
from pathlib import Path

data = []
for name in ['questions', 'hard', 'provocation']:
    path = Path(f'/kaggle/input/finetuneddata/{name}.json')
    if path.exists():
        with open(path, 'r', encoding='utf-8') as f:
            data.extend(json.load(f))

df = pd.DataFrame(data).sample(490)
df.columns = ['text', 'answer']
from datasets import Dataset

split_idx = int(len(df) * 0.9)
train_df = df.iloc[:split_idx].reset_index(drop=True)
val_df = df.iloc[split_idx:].reset_index(drop=True)

train_df = Dataset.from_pandas(train_df)
val_df = Dataset.from_pandas(val_df)


def format_chat_qwen3(example):
    messages_full = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": example["text"]},
        {"role": "assistant", "content": example["answer"]},
    ]
    
    # Применяем чат-шаблон один раз для получения полного диалога
    formatted_chat = tokenizer.apply_chat_template(
        messages_full, 
        tokenize=False,
        add_generation_prompt=False  # Важно: False для обучения
    )
    
    # Токенизируем один раз
    tokenized = tokenizer(formatted_chat)
    
    example["input_ids"] = tokenized["input_ids"]
    example["labels"] = tokenized["input_ids"].copy()  # Копируем input_ids как labels
    
    return example

# Применяем форматирование
train_df = train_df.map(format_chat_qwen3)
val_df = val_df.map(format_chat_qwen3)

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [16]:
desired_max_length = 512
tokenizer.model_max_length = desired_max_length

In [29]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from peft import LoraConfig # Предполагается, что lora_config — это экземпляр LoraConfig

# Создание TrainingArguments остается без изменений
training_arguments = TrainingArguments(
    output_dir="./lora_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=10,
    logging_steps=10,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
)
# Создание DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Инициализация SFTTrainer
# Все параметры из SFTConfig переносятся напрямую сюда
trainer = SFTTrainer(
    model=model,
    train_dataset=train_df,
    eval_dataset=val_df,
    peft_config=lora_config,
    args=training_arguments,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/441 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/49 [00:00<?, ? examples/s]

In [30]:
print("Начало тренировки")
train_result = trainer.train()
print("Конец тренировки")

Начало тренировки


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


KeyboardInterrupt: 

In [31]:
import os
checkpoints = [d for d in os.listdir("./lora_model") if d.startswith("checkpoint-")]
print("Доступные чекпоинты:", checkpoints)

import json

for ckpt in sorted(checkpoints):
    trainer_state_file = f"./lora_model/{ckpt}/trainer_state.json"
    if os.path.exists(trainer_state_file):
        with open(trainer_state_file) as f:
            state = json.load(f)
            # Найди последний eval_loss
            logs = state['log_history']
            eval_logs = [log for log in logs if 'eval_loss' in log]
            if eval_logs:
                last_eval = eval_logs[-1]
                print(f"{ckpt}: eval_loss = {last_eval['eval_loss']:.4f}")

Доступные чекпоинты: ['checkpoint-111']
checkpoint-111: eval_loss = 1.2998


In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(
    model,
    "/kaggle/working/lora_model/checkpoint-111",  # ← Твой лучший чекпоинт
    device_map="auto"
)

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Кто создал Маинкрафт?"}
]

# 2. Примените чат-шаблон
text = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
)

# 3. Токенизируйте с шаблоном
inputs = tokenizer(text, return_tensors="pt").to(model.device)
inputs = {k: v.half() if v.dtype == torch.float32 else v for k, v in inputs.items()}

# 4. Генерируйте
outputs = model.generate(
    **inputs,
    max_new_tokens=25,
    do_sample=True,          # ← ВКЛЮЧИТЬ СЭМПЛИНГ!
    repetition_penalty=1.15, # штраф за повторы (очень важно!)
)

# 5. Декодируйте и извлеките ответ
full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Полный ответ:", full_response)

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "В каком году опубликована поэма «Полтава»?"}
]

# 2. Примените чат-шаблон
text = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# 3. Токенизируйте с шаблоном
inputs = tokenizer(text, return_tensors="pt").to(model.device)

# 4. Генерируйте
# 4. Генерируйте
outputs = model.generate(
    **inputs,
    max_new_tokens=25,
    do_sample=True,          # ← ВКЛЮЧИТЬ СЭМПЛИНГ!
    temperature=0.7,         # добавить "творчества"
    top_p=0.9,               # nucleus sampling
    repetition_penalty=1.99, # штраф за повторы (очень важно!)
)

# 5. Декодируйте и извлеките ответ
full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("Полный ответ:", full_response)

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import gc

# 1. Объединяем и выгружаем LoRA
model = model.merge_and_unload()

# 2. Очищаем память
torch.cuda.empty_cache()
gc.collect()

# 3. ВАЖНО: Сначала сохраняем полную модель
model.save_pretrained("./lora_final_fp16", safe_serialization=True)
tokenizer.save_pretrained("./lora_final_fp16")